In [1]:
from utils import *
from dataset import *
from model import *
from classifier import *

In [2]:
from sklearn.utils.extmath import fast_logdet

def ebic(covariance, precision, n_samples, n_features, gamma=0):
    """
    Extended Bayesian Information Criteria for model selection.
    When using path mode, use this as an alternative to cross-validation for
    finding lambda.
    See:
        "Extended Bayesian Information Criteria for Gaussian Graphical Models"
        R. Foygel and M. Drton, NIPS 2010
    Parameters
    ----------
    covariance : 2D ndarray (n_features, n_features)
        Maximum Likelihood Estimator of covariance (sample covariance)
    precision : 2D ndarray (n_features, n_features)
        The precision matrix of the model to be tested
    n_samples :  int
        Number of examples.
    n_features : int
        Dimension of an example.
    gamma : (float) \in (0, 1)
        Choice of gamma=0 leads to classical BIC
        Positive gamma leads to stronger penalization of large graphs.
    Returns
    -------
    ebic score (float).  Caller should minimized this score.
    """
    l_theta = -np.sum(covariance * precision) + fast_logdet(precision)
    l_theta *= n_features / 2.

    # is something goes wrong with fast_logdet, return large value
    if np.isinf(l_theta) or np.isnan(l_theta):
        return 1e10

    mask = np.abs(precision.flat) > np.finfo(precision.dtype).eps
    precision_nnz = (np.sum(mask) - n_features) / 2.0  # lower off diagonal tri

    return (
        -2.0 * l_theta
        + precision_nnz * np.log(n_samples)
        + 4.0 * precision_nnz * np.log(n_features) * gamma
    )

In [3]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)  

In [7]:
%%time

# Model - SBM, noise = 0.4, diffusion_coef = 0.2, 

dataset = Dataset(tag='EXP')

scaler = StandardScaler()

from sklearn.model_selection import KFold

NrFolds = 4
kf = KFold(n_splits=NrFolds)

alpha_grid = np.linspace(0.0, 3.0, num=150)

EBIC = np.zeros(len(alpha_grid))

n_obs_grid = [100, 500, 1000]
n_features_grid = [50, 125, 250]
signal_grid = [2, 5, 10]

# n_obs_grid = [100]
# n_features_grid = [50]
# signal_grid = [2]

alpha_optim_grid = []

for n_obs_s in n_obs_grid:
    
    for n_features_s in n_features_grid: 
        
        for signal_s in signal_grid: 

            dataset.create_syn(n_classes = 5, 
                               n_obs_train = n_obs_s, 
                               n_obs_test= n_obs_s, 
                               n_features=n_features_s,
                               n_char_features = 10, 
                               signal =[signal_s, signal_s], 
                               diff_coef=[.02, .02], 
                               noise = [.4, .4], 
                               n_communities = 5,
                               probs = [0.9, 0.1], 
                               n_iter=1, 
                               model ='SBM')
            
            data_s = scaler.fit_transform(dataset.X_train)
            
            data = data_s
            
            for idx, alphas in enumerate(alpha_grid, start=0):
                
                for train_index, test_index in kf.split(data):
                    
                    X_trn, X_tst = data[train_index,:], data[test_index,:]
    
                    _ , n_samples_trn = X_trn.shape
                    cov_emp_trn = np.dot(X_trn.T, X_trn) / n_samples_trn
     
                    covariance_trn, precision_matrix_trn = graphical_lasso(emp_cov=cov_emp_trn, alpha=alphas, mode='cd')
    
                    n_features_tst, n_samples_tst = X_tst.shape
                    cov_emp_tst = np.dot(X_tst.T, X_tst) / n_samples_tst
    
                    covariance_tst, precision_matrix_tst = graphical_lasso(emp_cov=cov_emp_tst, alpha=alphas, mode='cd')
    
                    EBIC[idx] = EBIC[idx] + ebic(covariance_trn, precision_matrix_tst, n_samples_tst, n_features_tst, gamma = 0)
 
             
            index_min_ebic = np.argmin(EBIC)

            alpha_optim_ebic = alpha_grid[index_min_ebic]

            alpha_optim_grid = np.append(alpha_optim_grid, alpha_optim_ebic)  

Wall time: 1h 28min 24s


In [9]:
alpha_optim_ebic

2.919463087248322

In [8]:
alpha_optim_grid

array([0.84563758, 0.88590604, 0.90604027, 0.90604027, 0.90604027,
       0.90604027, 0.90604027, 0.        , 0.90604027, 2.15436242,
       2.17449664, 2.25503356, 2.25503356, 2.25503356, 2.25503356,
       2.25503356, 2.25503356, 2.25503356, 2.91946309, 2.91946309,
       2.91946309, 2.91946309, 2.91946309, 2.91946309, 2.91946309,
       2.91946309, 2.91946309])